# How to Train Your Own Cone Detection and Key Points Detection Networks

![](https://user-images.githubusercontent.com/22118253/70957091-fe06a480-2042-11ea-8c06-0fcc549fc19a.png)

In this notebook, we will demonstrate 
- how to train your own YOLOv3-based traffic cone detection network and do inference on a video.
- how to train your own KeyPoints detection network and do inference on pictures of traffic cone.

**[Accurate Low Latency Visual Perception for Autonomous Racing: Challenges Mechanisms and Practical Solutions](https://github.com/mit-han-lab/once-for-all)** is an accurate low latency visual perception system introduced by Kieran Strobel, Sibo Zhu, Raphael Chang, and Skanda Koppula.

![](https://user-images.githubusercontent.com/22118253/70950893-e2de6980-202f-11ea-9a16-399579926ee5.gif)

## 1. Preparation
Let's first install all the required packages:

In [1]:
! sudo apt install unzip
print('Installing PyTorch...')
! pip3 install torch 
print('Installing torchvision...')
! pip3 install torchvision 
print('Installing numpy...')
! pip3 install numpy 
# tqdm is a package for displaying a progress bar.
print('Installing tqdm (progress bar) ...')
! pip3 install tqdm 
print('Installing matplotlib...')
! pip3 install matplotlib 
print('Installing all the other required packages once for all')
! sudo python3 setup.py install




unzip is already the newest version (6.0-21ubuntu1).
The following packages were automatically installed and are no longer required:
  cuda-10-1 cuda-command-line-tools-10-1 cuda-compiler-10-1 cuda-cudart-10-1
  cuda-cudart-dev-10-1 cuda-cufft-10-1 cuda-cufft-dev-10-1 cuda-cuobjdump-10-1
  cuda-cupti-10-1 cuda-curand-10-1 cuda-curand-dev-10-1 cuda-cusolver-10-1
  cuda-cusolver-dev-10-1 cuda-cusparse-10-1 cuda-cusparse-dev-10-1
  cuda-demo-suite-10-1 cuda-documentation-10-1 cuda-driver-dev-10-1
  cuda-gdb-10-1 cuda-gpu-library-advisor-10-1 cuda-libraries-10-1
  cuda-libraries-dev-10-1 cuda-license-10-1 cuda-memcheck-10-1
  cuda-misc-headers-10-1 cuda-npp-10-1 cuda-npp-dev-10-1 cuda-nsight-10-1
  cuda-nsight-compute-10-1 cuda-nsight-systems-10-1 cuda-nvcc-10-1
  cuda-nvdisasm-10-1 cuda-nvgraph-10-1 cuda-nvgraph-dev-10-1 cuda-nvjpeg-10-1
  cuda-nvjpeg-dev-10-1 cuda-nvml-dev-10-1 cuda-nvprof-10-1 cuda-nvprune-10-1
  cuda-nvrtc-10-1 cuda-nvrtc-dev-10-1 cuda-nvtx-10-1 cuda-nvvp-10-1
  cud

Before we start training, let's download the Cone Detection dataset and the corresponding label and intial training weights. 

In [ ]:
print("Downloading Training Dataset")
! wget https://storage.googleapis.com/mit-driverless-open-source/YOLO_Dataset.zip
! unzip YOLO_Dataset.zip
! mv YOLO_Dataset dataset/ && rm YOLO_Dataset.zip
print("Downloading YOLOv3 Sample Weights")
! wget https://storage.googleapis.com/mit-driverless-open-source/yolov3-training/sample-yolov3.weights 
print("Downloading Training and Validation Label")
! cd dataset/ && wget https://storage.googleapis.com/mit-driverless-open-source/yolov3-training/all.csv && cd ..
! cd dataset/ && wget https://storage.googleapis.com/mit-driverless-open-source/yolov3-training/train.csv && cd ..
! cd dataset/ && wget https://storage.googleapis.com/mit-driverless-open-source/yolov3-training/validate.csv && cd ..

## 2. Using Pretrained YOLOv3 Weights File to Start Training


First, import all the packages used in this tutorial:

In [ ]:
import argparse
import os
import random
import tempfile
import time
import multiprocessing
import subprocess
import math
import shutil
import math

from datetime import datetime
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from models import Darknet
from utils.datasets import ImageLabelDataset
from utils.utils import model_info, print_args, Logger, visualize_and_save_to_local,xywh2xyxy
import validate
import warnings
import sys

##### section for all random seeds #####
torch.manual_seed(17)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
########################################

warnings.filterwarnings("ignore")
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

cuda = torch.cuda.is_available()
device = torch.device('cuda:0' if cuda else 'cpu')
num_cpu = multiprocessing.cpu_count() if cuda else 0
if cuda:
    torch.cuda.synchronize()
random.seed(0)
torch.manual_seed(0)

if cuda:
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)
    torch.backends.cudnn.benchmark = True
    torch.cuda.empty_cache()

Successfully imported all packages and configured random seed to 17!

In [ ]:
def run_epoch(label_prefix, data_loader, num_steps, optimizer, model, epoch, num_epochs, step):
    print(f"Model in {label_prefix} mode")
    epoch_losses = [0.0] * 7
    epoch_time_total = 0.0
    epoch_num_targets = 1e-12
    t1 = time.time()
    loss_labels = ["Total", "L-x", "L-y", "L-w", "L-h", "L-noobj", "L-obj"]
    for i, (img_uri, imgs, targets) in enumerate(data_loader):
        if step[0] >= num_steps:
            break
        imgs = imgs.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)
        targets.requires_grad_(False)
        step_num_targets = ((targets[:, :, 1:5] > 0).sum(dim=2) > 1).sum().item() + 1e-12
        epoch_num_targets += step_num_targets
        # Compute loss, compute gradient, update parameters
        if optimizer is not None:
            optimizer.zero_grad()
        losses = model(imgs, targets)
        if label_prefix == "train":
            losses[0].sum().backward()
        if optimizer is not None:
            optimizer.step()

        for j, (label, loss) in enumerate(zip(loss_labels, losses)):
            batch_loss = loss.sum().to('cpu').item()
            epoch_losses[j] += batch_loss
        finished_time = time.time()
        step_time_total = finished_time - t1
        epoch_time_total += step_time_total
        
        statement = label_prefix + ' Epoch: ' + str(epoch) + ', Batch: ' + str(i + 1) + '/' + str(len(data_loader))
        count = 0
        for (loss_label, loss) in zip(loss_labels, losses):
            if count == 0:
                statement += ', Total: ' + '{0:10.6f}'.format(loss.item() / step_num_targets)
                tot_loss = loss.item()
                count += 1
            else:
                statement += ',   ' + loss_label + ': {0:5.2f}'.format(loss.item() / tot_loss * 100) + '%'
        print(statement)
        if label_prefix == "train":
            step[0] += 1
    return epoch_losses, epoch_time_total, epoch_num_targets

Training Config

In [ ]:
evaluate = False
batch_size = int(5)
optimizer_pick = "Adam"
model_cfg = "model_cfg/yolo_baseline.cfg"
weights_path = "sample-yolov3.weights"
output_path = "automatic"
dataset_path = "dataset/YOLO_Dataset/"
num_epochs = int(2048)
num_steps = 8388608
checkpoint_interval = int(1)
augment_affine = False
augment_hsv = False
lr_flip = False
ud_flip = False
momentum = float(0.9)
gamma = float(0.95)
lr = float(0.001)
weight_decay = float(0.0)
vis_batch = int(0)
data_aug = False
blur = False
salt = False
noise = False
contrast = False
sharpen = False
ts = True
debug_mode = False
upload_dataset = False
xy_loss = float(2)
wh_loss= float(1.6)
no_object_loss = float(25)
object_loss = float(0.1)
vanilla_anchor = False
val_tolerance = int(3)
min_epochs = int(3)

In [ ]:
input_arguments = list(locals().items())

print("Initializing model")
model = Darknet(config_path=model_cfg,xy_loss=xy_loss,wh_loss=wh_loss,no_object_loss=no_object_loss,object_loss=object_loss,vanilla_anchor=vanilla_anchor)
img_width, img_height = model.img_size()
bw  = model.get_bw()
validate_uri, train_uri = model.get_links()

if output_path == "automatic":
    current_month = datetime.now().strftime('%B').lower()
    current_year = str(datetime.now().year)
    if not os.path.exists(os.path.join('outputs/', current_month + '-' + current_year + '-experiments/' + model_cfg.split('.')[0].split('/')[-1])):
        os.makedirs(os.path.join('outputs/', current_month + '-' + current_year + '-experiments/' + model_cfg.split('.')[0].split('/')[-1]))
    output_uri = os.path.join('outputs/', current_month + '-' + current_year + '-experiments/' + model_cfg.split('.')[0].split('/')[-1])
else:
    output_uri = output_path

num_validate_images, num_train_images = model.num_images()
conf_thresh, nms_thresh, iou_thresh = model.get_threshs()
num_classes = model.get_num_classes()
loss_constant = model.get_loss_constant()
conv_activation = model.get_conv_activation()
anchors = model.get_anchors()
onnx_name = model.get_onnx_name()

with tempfile.TemporaryDirectory() as tensorboard_data_dir:
    print("Initializing data loaders")
    train_data_loader = torch.utils.data.DataLoader(
        ImageLabelDataset(train_uri, dataset_path=dataset_path, width=img_width, height=img_height, augment_hsv=augment_hsv,
                            augment_affine=augment_affine, num_images=num_train_images,
                            bw=bw, n_cpu=num_cpu, lr_flip=lr_flip, ud_flip=ud_flip,vis_batch=vis_batch,data_aug=data_aug,blur=blur,salt=salt,noise=noise,contrast=contrast,sharpen=sharpen,ts=ts,debug_mode=debug_mode, upload_dataset=upload_dataset),
        batch_size=(1 if debug_mode else batch_size),
        shuffle=(False if debug_mode else True),
        num_workers=(0 if vis_batch else num_cpu),
        pin_memory=cuda)
    print("Num train images: ", len(train_data_loader.dataset))

    validate_data_loader = torch.utils.data.DataLoader(
        ImageLabelDataset(validate_uri, dataset_path=dataset_path, width=img_width, height=img_height, augment_hsv=False,
                            augment_affine=False, num_images=num_validate_images,
                            bw=bw, n_cpu=num_cpu, lr_flip=False, ud_flip=False,vis_batch=vis_batch,data_aug=False,blur=False,salt=False,noise=False,contrast=False,sharpen=False,ts=ts,debug_mode=debug_mode, upload_dataset=upload_dataset),
        batch_size=(1 if debug_mode else batch_size),
        shuffle=False,
        num_workers=(0 if vis_batch else num_cpu),
        pin_memory=cuda)
    print("Num validate images: ", len(validate_data_loader.dataset))

    ##### additional configuration #####
    print("Training batch size: " + str(batch_size))
    
    print("Checkpoint interval: " + str(checkpoint_interval))

    print("Loss constants: " + str(loss_constant))

    print("Anchor boxes: " + str(anchors))

    print("Training image width: " + str(img_width))

    print("Training image height: " + str(img_height))

    print("Confidence Threshold: " + str(conf_thresh))

    print("Number of training classes: " + str(num_classes))

    print("Conv activation type: " + str(conv_activation))

    print("Starting learning rate: " + str(lr))

    if ts:
        print("Tile and scale mode [on]")
    else:
        print("Tile and scale mode [off]")

    if data_aug:
        print("Data augmentation mode [on]")
    else:
        print("Data augmentation mode [off]")

    ####################################

    start_epoch = 0

    weights_path = weights_path
    if optimizer_pick == "Adam":
        print("Using Adam Optimizer")
        optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),
                                    lr=lr, weight_decay=weight_decay)
    elif optimizer_pick == "SGD":
        print("Using SGD Optimizer")
        optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()),
                                lr=lr, momentum=momentum, weight_decay=weight_decay)
    else:
        raise Exception(f"Invalid optimizer name: {optimizer_pick}")
    print("Loading weights")
    model.load_weights(weights_path, model.get_start_weight_dim())

    if torch.cuda.device_count() > 1:
        print('Using ', torch.cuda.device_count(), ' GPUs')
        model = nn.DataParallel(model)
    model = model.to(device, non_blocking=True)

    # Set scheduler
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=gamma)

    val_loss = 999  # using a high number for validation loss
    val_loss_counter = 0
    step = [0]  # wrapping in an array so it is mutable
    epoch = start_epoch
    while epoch < num_epochs and step[0] < num_steps and not evaluate:
        epoch += 1
        scheduler.step()
        model.train()
        run_epoch(label_prefix="train", data_loader=train_data_loader, epoch=epoch,
                    step=step, model=model, num_epochs=num_epochs, num_steps=num_steps,
                    optimizer=optimizer)
        print('Completed epoch: ', epoch)
        # Update best loss
        if epoch % checkpoint_interval == 0 or epoch == num_epochs or step[0] >= num_steps:
            # First, save the weights
            save_weights_uri = os.path.join(output_uri, "{epoch}.weights".format(epoch=epoch))
            model.save_weights(save_weights_uri)

            with torch.no_grad():
                print("Calculating loss on validate data")
                epoch_losses, epoch_time_total, epoch_num_targets = run_epoch(
                    label_prefix="validate", data_loader=validate_data_loader, epoch=epoch,
                    model=model, num_epochs=num_epochs, num_steps=num_steps, optimizer=None,
                    step=step)
                avg_epoch_loss = epoch_losses[0] / epoch_num_targets
                print('Average Validation Loss: {0:10.6f}'.format(avg_epoch_loss))

                if avg_epoch_loss > val_loss and epoch > min_epochs:
                    val_loss_counter += 1
                    print(f"Validation loss did not decrease for {val_loss_counter}"
                            f" consecutive check(s)")
                else:
                    print("Validation loss decreased. Yay!!")
                    val_loss_counter = 0
                    val_loss = avg_epoch_loss
                    ##### updating best result for optuna study #####
                    result = open("logs/result.txt", "w" )
                    result.write(str(avg_epoch_loss))
                    result.close() 
                    ###########################################
                validate.validate(dataloader=validate_data_loader, model=model, device=device, step=step[0], bbox_all=False,debug_mode=debug_mode)
                if val_loss_counter == val_tolerance:
                    print("Validation loss stopped decreasing over the last " + str(val_tolerance) + " checkpoints, creating onnx file")
                    with tempfile.NamedTemporaryFile() as tmpfile:
                        model.save_weights(tmpfile.name)
                        weights_name = tmpfile.name
                        cfg_name = os.path.join(tempfile.gettempdir(), model_cfg.split('/')[-1].split('.')[0] + '.tmp')
                        onnx_gen = subprocess.call(['python3', 'yolo2onnx.py', '--cfg_name', cfg_name, '--weights_name', weights_name])
                        save_weights_uri = os.path.join(output_uri, onnx_name)
                        os.rename(weights_name, save_weights_uri)
                        try:
                            os.remove(onnx_name)
                        except:
                            pass
                        os.remove(cfg_name)
                    break
    if evaluate:
        validation = validate.validate(dataloader=validate_data_loader, model=model, device=device, step=-1, bbox_all=False, tensorboard_writer=None,debug_mode=debug_mode)
return val_loss


## 3. Inference